In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# lA ID de la GPU a usar, puede ser desde 0 hasta las N GPU's. Si es -1 significa que es en la CPU
os.environ["CUDA_VISIBLE_DEVICES"]="0";

# Importacion de librerias

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import warnings
warnings.filterwarnings("ignore")

import argparse
import tensorflow as tf
import random
import IPython.display as display
from random import randint

# Configuraciones de TensorFlow

### Configuracion de TensorFlow 2.0

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

### Configuracion de TensorFlow 1.0

In [3]:
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True

### Modo eagerly

In [4]:
#Comprobar que estoy ejecutandome en modo eagerly
tf.executing_eagerly()

False

# Carga de Datos

In [5]:
root_path = "/home/jefelitman/DataSets/ucf101/split_1"
root_path

'/home/jefelitman/DataSets/ucf101/split_1'

In [6]:
batch_size = 10
size = [112,112]
frames = 16
canales = 3

In [7]:
import sys
sys.path.append('../')

from Datasets_utils.DatasetsLoader import UCF101

dataset = UCF101(root_path,10,True)

In [ ]:
images_placeholder = tf.placeholder(tf.float32, shape=(batch_size,
                                                         c3d_model.NUM_FRAMES_PER_CLIP,
                                                         c3d_model.CROP_SIZE,
                                                         c3d_model.CROP_SIZE,
                                                         c3d_model.CHANNELS))
  labels_placeholder = tf.placeholder(tf.int64, shape=(batch_size))

In [ ]:
dataset.batch_index

# Red Neuronal

In [8]:
from LTC_v2 import LTC

Sesion = tf.Session(config=config)

ltc = LTC(101,batch_size,[16,112,112,3])

W1013 22:57:01.337800 140079860377408 deprecation_wrapper.py:119] From /home/jefelitman/Biv2LabNN/LTC/LTC_v2.py:15: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



In [9]:
ltc.inicializar_modelo(0.1,0.1)
init = tf.global_variables_initializer()
Sesion.run(init)

W1013 22:57:04.109581 140079860377408 deprecation_wrapper.py:119] From /home/jefelitman/Biv2LabNN/LTC/LTC_v2.py:28: The name tf.train.get_or_create_global_step is deprecated. Please use tf.compat.v1.train.get_or_create_global_step instead.

W1013 22:57:04.115672 140079860377408 deprecation_wrapper.py:119] From /home/jefelitman/Biv2LabNN/LTC/LTC_v2.py:117: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W1013 22:57:04.116565 140079860377408 deprecation_wrapper.py:119] From /home/jefelitman/Biv2LabNN/LTC/LTC_v2.py:118: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W1013 22:57:04.117443 140079860377408 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype 

ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ["<tf.Variable 'conv1/W:0' shape=(3, 3, 3, 3, 64) dtype=float32_ref>", "<tf.Variable 'conv2/W:0' shape=(3, 3, 3, 64, 128) dtype=float32_ref>", "<tf.Variable 'conv3/W:0' shape=(3, 3, 3, 128, 256) dtype=float32_ref>", "<tf.Variable 'conv4/W:0' shape=(3, 3, 3, 256, 256) dtype=float32_ref>", "<tf.Variable 'conv5/W:0' shape=(3, 3, 3, 256, 256) dtype=float32_ref>", "<tf.Variable 'fc6/W:0' shape=(4096, 2048) dtype=float32_ref>", "<tf.Variable 'fc6/B:0' shape=(2048,) dtype=float32_ref>", "<tf.Variable 'fc7/W:0' shape=(2048, 2048) dtype=float32_ref>", "<tf.Variable 'fc7/B:0' shape=(2048,) dtype=float32_ref>", "<tf.Variable 'fc8/W:0' shape=(2048, 101) dtype=float32_ref>", "<tf.Variable 'fc8/B:0' shape=(101,) dtype=float32_ref>"] and loss Tensor("costo/Mean:0", shape=(), dtype=float32).

In [ ]:
for i in range(dataset.train_batches):
    print("Batch: ",i)
    batch, labels = dataset.get_next_train_batch([112,112],16)
    labels = tf.cast(labels, dtype=tf.int64)
    Sesion.run(ltc.entrenar(batch,labels,0.5,0.1))

In [10]:
tf.trainable_variables()

[<tf.Variable 'conv1/W:0' shape=(3, 3, 3, 3, 64) dtype=float32_ref>,
 <tf.Variable 'conv2/W:0' shape=(3, 3, 3, 64, 128) dtype=float32_ref>,
 <tf.Variable 'conv3/W:0' shape=(3, 3, 3, 128, 256) dtype=float32_ref>,
 <tf.Variable 'conv4/W:0' shape=(3, 3, 3, 256, 256) dtype=float32_ref>,
 <tf.Variable 'conv5/W:0' shape=(3, 3, 3, 256, 256) dtype=float32_ref>,
 <tf.Variable 'fc6/W:0' shape=(4096, 2048) dtype=float32_ref>,
 <tf.Variable 'fc6/B:0' shape=(2048,) dtype=float32_ref>,
 <tf.Variable 'fc7/W:0' shape=(2048, 2048) dtype=float32_ref>,
 <tf.Variable 'fc7/B:0' shape=(2048,) dtype=float32_ref>,
 <tf.Variable 'fc8/W:0' shape=(2048, 101) dtype=float32_ref>,
 <tf.Variable 'fc8/B:0' shape=(101,) dtype=float32_ref>]

In [ ]:
from LTC_v2 import LTC

model = LTC(101):

logging_hook = tf.train.LoggingTensorHook(
      tensors={'step': model.global_step,
               'loss': model.cost,
               'precision': precision},
      every_n_iter=100)

training_session = tf.train.MonitoredTrainingSession(
    checkpoint_dir="./Checkpoints",
      hooks=[logging_hook, _LearningRateSetterHook()],
      chief_only_hooks=[summary_hook],
      # Since we provide a SummarySaverHook, we need to disable default
      # SummarySaverHook. To do that we set save_summaries_steps to 0.
      save_summaries_steps=0,
      config=config
)


In [ ]:
https://www.math.purdue.edu/~nwinovic/slides/Getting_Started_with_TensorFlow_II.pdf